In [1]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm import tqdm
# from openslide import OpenSlide

import torch
from torch import nn
from torch.utils.data import (
    ConcatDataset,
    DataLoader,
    Dataset,
    Subset,
    SubsetRandomSampler,
    TensorDataset,
    random_split,
)

import torchvision
from torchvision import transforms
from PIL import Image

import einops

# from eval_metrics import print_metrics_regression
from sklearn import metrics as sklearn_metrics

In [2]:
train = pd.read_pickle("./datasets/train.pkl")
train_x = train["x"]
train_y = train["y"]
train_x = torch.tensor(torch.stack(train_x).detach().cpu().numpy())
train_y = torch.tensor(train_y)

test = pd.read_pickle("./datasets/test.pkl")
test_x = test["x"]
test_y = test["y"]
test_x = torch.tensor(torch.stack(test_x).detach().cpu().numpy())
test_y = torch.tensor(test_y)

In [3]:
min_label = train_y.min().item()
max_label = train_y.max().item()
train_y = (train_y-min_label)/(max_label-min_label)
test_y = (test_y-min_label)/(max_label-min_label)

In [4]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

(torch.Size([5073, 3, 224, 224]),
 torch.Size([5073]),
 torch.Size([6301, 3, 224, 224]),
 torch.Size([6301]))

In [5]:
class ImageDataset(Dataset):
    def __init__(self, x, y):
        self.x = x # img_tensor_list
        self.y = y # label

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return len(self.x)

In [13]:
batch_size = 64

epochs = 100
learning_rate = 1e-3
momentum = 0.9
weight_decay=0 # 1e-8

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [14]:
train_dataset = ImageDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_dataset = ImageDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [15]:
# for data in train_dataset:
#     x, y = data
#     print(x.shape, y)

In [16]:
def mse_loss(y_pred, y_true):
    loss_fn = nn.MSELoss()
    return loss_fn(y_pred, y_true)

def focal_mse_loss(inputs, targets, activate='sigmoid', beta=.2, gamma=1):
    loss = (inputs - targets) ** 2
    loss *= (torch.tanh(beta * torch.abs(inputs - targets))) ** gamma if activate == 'tanh' else \
        (2 * torch.sigmoid(beta * torch.abs(inputs - targets)) - 1) ** gamma
    loss = torch.mean(loss)
    return loss

def huber_loss(inputs, targets, beta=1.):
    l1_loss = torch.abs(inputs - targets)
    cond = l1_loss < beta
    loss = torch.where(cond, 0.5 * l1_loss ** 2 / beta, l1_loss - 0.5 * beta)
    loss = torch.mean(loss)
    return loss

criterion = mse_loss

In [17]:
def train_epoch(model, dataloader, loss_fn, optimizer, scheduler):
    train_loss = []
    model.train()
    for step, data in enumerate(dataloader):
        batch_x, batch_y = data
        batch_x, batch_y = (
            batch_x.float().to(device),
            batch_y.float().to(device),
        )
        optimizer.zero_grad()
        # print(batch_x.device, batch_x.shape)
        # print(next(model.parameters()).is_cuda)
        output = model(batch_x)
        output = torch.squeeze(output, dim=1)
        # print(output.shape, batch_y.shape)
        
        loss = loss_fn(output, batch_y)
        train_loss.append(loss.item())
        loss.backward()
        optimizer.step()
    metric_train_loss = np.array(train_loss).mean()
    scheduler.step(metric_train_loss)
    return metric_train_loss

def val_epoch(model, dataloader):
    y_pred = []
    y_true = []
    model.eval()
    with torch.no_grad():
        for step, data in enumerate(dataloader):
            # print(step)
            batch_x, batch_y = data
            batch_x, batch_y = (
                batch_x.float().to(device),
                batch_y.float().to(device),
            )
            output = model(batch_x)
            output = torch.squeeze(output, dim=1)
            y_pred.extend(output.detach().cpu().numpy().tolist())
            y_true.extend(batch_y.detach().cpu().numpy().tolist())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    y_pred = y_pred*(max_label-min_label)+min_label
    y_true = y_true*(max_label-min_label)+min_label
    mse = sklearn_metrics.mean_squared_error(y_true, y_pred)
    return mse

In [18]:
model = torchvision.models.resnet18(num_classes=1)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
hidden_dim = model.fc.in_features
out_dim = 1

model.fc = nn.Sequential(
    nn.Linear(hidden_dim, hidden_dim//4),
    nn.GELU(),
    nn.Linear(hidden_dim//4, out_dim),
    nn.Sigmoid()
)

model.load_state_dict(torch.load('./checkpoints/resnet18-f37072fd.pth'), strict=False)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
best_score = 1e8
for epoch in range(epochs):
    # print(f'Running epoch {epoch} ...')
    train_loss = train_epoch(
        model,
        train_loader,
        criterion,
        optimizer,
        scheduler
    )
    metric_valid = val_epoch(model, test_loader)
    if metric_valid < best_score:
        best_score = metric_valid
        print("Saving best model ...")
        print("Val Score:", metric_valid)
        torch.save(
            model.state_dict(),
            f"./checkpoints/model.ckpt",
        )
    print(f"Epoch {epoch}: Loss = {train_loss}")

Saving best model ...
Val Score: 0.682558129468998
Epoch 0: Loss = 0.052732481139537414
Saving best model ...
Val Score: 0.67335196080321
Epoch 1: Loss = 0.04741968618473038
Epoch 2: Loss = 0.0464678766351426
Epoch 3: Loss = 0.04605944804061437
Epoch 4: Loss = 0.04634224193505361
Epoch 5: Loss = 0.04621467228062102
Epoch 6: Loss = 0.04604455318840337
Epoch 7: Loss = 0.045892055430886106
Epoch 8: Loss = 0.04575770124574774
Epoch 9: Loss = 0.04516891814710107
Epoch 10: Loss = 0.04545648777420865
Epoch 11: Loss = 0.044942271804029585
Epoch 12: Loss = 0.04494224065274466
Epoch 13: Loss = 0.045159774340572764
Epoch 14: Loss = 0.04441595063108252
Epoch 15: Loss = 0.04519156152382493
Epoch 16: Loss = 0.04409004190092673
Epoch 17: Loss = 0.043118219610187226
Epoch 18: Loss = 0.04401836885226658
Epoch 19: Loss = 0.04243440712016309
Epoch 20: Loss = 0.041249737501493654
Epoch 21: Loss = 0.04145061722374521
Saving best model ...
Val Score: 0.66873327894979
Epoch 22: Loss = 0.04148063597822329
Epo

In [26]:
best_score

0.6680583233472928